In [ ]:
from pymongo import MongoClient
from pymongo.encryption import (ClientEncryption, AutoEncryptionOpts, Algorithm, QueryType)
from bson.codec_options import CodecOptions
from bson.binary import STANDARD
from dotenv import load_dotenv
import os
import pprint
import datetime

In [ ]:
# setup some variables
load_dotenv()
uri = os.environ['MONGODB_URI']
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault_customers_ex"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "sample_analytics"
encrypted_collection_name = "customers_enc_ex"

In [ ]:
# build local kms provider credentials with the local CMK
try:
    path = "./customer-master-key.txt"
    with open(path, "rb") as f:
        local_master_key = f.read()
        kms_provider_credentials = {
            "local": {
                "key": local_master_key
            },
        }
except Exception as e:
    raise Exception("Unable to read Customer Master Key from file due to the following error: ", e)

# build MongoDB clients (standard one and encrypted)
encrypted_client = MongoClient(
    uri,
    auto_encryption_opts= AutoEncryptionOpts(
        kms_provider_credentials,
        key_vault_namespace,
        bypass_query_analysis=True
    )
)

standard_client = MongoClient(
    uri)

# get the encrypted collection with the two clients
encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]
collection = standard_client[encrypted_database_name][encrypted_collection_name]

In [ ]:
# build the ClientEncryption helper
client_encryption = ClientEncryption(
    kms_provider_credentials,
    key_vault_namespace,
    encrypted_client,
    CodecOptions(uuid_representation=STANDARD),
)

In [ ]:
# retrieve all the DEK
key_vault = standard_client[key_vault_database_name][key_vault_collection_name]

data_key_name_id = key_vault.find_one({"keyAltNames": "dataKey1"})["_id"]
data_key_active_id = key_vault.find_one({"keyAltNames": "dataKey2"})["_id"]
data_key_accounts_id = key_vault.find_one({"keyAltNames": "dataKey3"})["_id"]
data_key_address_id = key_vault.find_one({"keyAltNames": "dataKey4"})["_id"]
data_key_email_id = key_vault.find_one({"keyAltNames": "dataKey5"})["_id"]
data_key_birthdate_id = key_vault.find_one({"keyAltNames": "dataKey6"})["_id"]

### Insert document with unconfigured client

In [ ]:
try:
    collection.insert_one({
        'accounts': [550665, 321695],
        'address': '20 rue Quentin Bauchart\n 75008 PARIS',
        'birthdate': datetime.datetime(1978, 5, 11, 0, 0),
        'email': 'john.doe@mongodb.com',
        'name': 'John Doe',
        'tier_and_details': {},
        'username': 'whoami'
    })
except Exception as e:
    print("Unable to insert document (as expected) with error: ", e)

### Insert document

In [ ]:
# encrypt each needed fields
encryptedName = client_encryption.encrypt(
    "John Doe", Algorithm.INDEXED, data_key_name_id, contention_factor=1)

encryptedAccounts =  client_encryption.encrypt(
    [550665, 321695], Algorithm.UNINDEXED, data_key_accounts_id)

encryptedAddress =  client_encryption.encrypt(
    '20 rue Quentin Bauchart\n 75008 PARIS', Algorithm.UNINDEXED, data_key_address_id)

encryptedEmail = client_encryption.encrypt(
    'john.doe@mongodb.com', Algorithm.INDEXED, data_key_email_id, contention_factor=1)

encryptedBirthdate = client_encryption.encrypt(
    datetime.datetime(1978, 5, 11, 0, 0), Algorithm.INDEXED, data_key_birthdate_id, contention_factor=1)

# insert document
encrypted_collection.insert_one({
    'accounts': encryptedAccounts,
    'address': encryptedAddress,
    'birthdate': encryptedBirthdate,
    'email': encryptedEmail,
    'name': encryptedName,
    'tier_and_details': {},
    'username': 'whoami'
})

### Query on unencrypted String field (with unconfigured client)

In [ ]:
find_result = collection.find_one({
    "username": "whoami"
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on encrypted String field (with unconfigured client)

In [ ]:
find_result = collection.find_one({
    "name": "John Doe"
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on encrypted String field

In [ ]:
# encrypt the query field
encryptedName = client_encryption.encrypt(
    "John Doe",
    Algorithm.INDEXED,
    data_key_name_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

# execute the query
find_result = encrypted_collection.find_one({
    "name": encryptedName
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on Date field

In [ ]:
# encrypt the query field
encryptedBirthdate = client_encryption.encrypt(
    datetime.datetime(1988, 6, 20, 0, 0),
    Algorithm.INDEXED,
    data_key_birthdate_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

# execute the query
find_result = encrypted_collection.find_one({
   "birthdate": encryptedBirthdate 
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on Boolean field with $in operator

In [ ]:
# encrypt the query field
encryptedActive = client_encryption.encrypt(
    True,
    Algorithm.INDEXED,
    data_key_active_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

# execute the query
find_result = encrypted_collection.find_one({
  "active": { "$in": [ encryptedActive ] } 
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on unencrypted and encrypted String fields ($and)

In [ ]:
# encrypt the query field
encryptedName = client_encryption.encrypt(
    "John Doe",
    Algorithm.INDEXED,
    data_key_name_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

# execute the query
find_result = encrypted_collection.find_one({
    "$and" : [ { "name": encryptedName }, { "username": "whoami" } ]
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on two encrypted String fields ($and)

In [ ]:
# encrypt the query fields
encryptedName = client_encryption.encrypt(
    "John Doe",
    Algorithm.INDEXED,
    data_key_name_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

encryptedEmail = client_encryption.encrypt(
    "john.doe@mongodb.com",
    Algorithm.INDEXED,
    data_key_email_id,
    query_type=QueryType.EQUALITY,
    contention_factor=1
)

# execute the query
find_result = encrypted_collection.find_one({
    "$and" : [ { "name": encryptedName }, { "email": encryptedEmail } ]
},
{"__safeContent__": 0})
pprint.pprint(find_result)